# Spaceship Titanic

- [Link to Source](https://www.kaggle.com/competitions/spaceship-titanic/overview)

---

In [3]:
import sys

!{sys.executable} -m pip install hyperopt --quiet
!{sys.executable} -m pip install xgboost --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [50]:
import os
import time

import sagemaker
import time
import boto3
import numpy as np
import pandas as pd
from numpy import array
import xgboost as xgb

from sklearn.metrics import accuracy_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [51]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

bucket = "test-sagemaker-examples-1357942113492"
prefix = "DEMO_Spaceship"

In [52]:
train = pd.read_csv(f's3://{bucket}/{prefix}/train.csv')
test = pd.read_csv(f's3://{bucket}/{prefix}/test.csv')

In [53]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [54]:
def scale_columns(df, column):
    df[column] = df[column].fillna(df[column].mean())

    df[column] = (df[column].astype(int)-df[column].min())/(df[column].max() - df[column].min())
    return df[column]

def change_to_int(column_data, nan_value):
    column_data= column_data.fillna(nan_value)
    column_data= column_data.astype(int)
    return column_data

def one_hot_encode(df, column):
    df[column].fillna(df[column].value_counts().idxmax(), inplace=True)
    
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(df[column])
    num_unique_val = len(set(integer_encoded))
    
    hot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    hot_encoded = hot_encoder.fit_transform(integer_encoded)
    return hot_encoded, num_unique_val

In [55]:
def dataset_transform(df, split_target):
    if "Transported" in df.columns:
        first_col = df.pop('Transported').astype(int)
        df.insert(0, 'Transported', first_col)
    
    passenger_id = df['PassengerId']
       
    df['VIP'] = change_to_int(df['VIP'], False)
    df['CryoSleep'] = change_to_int(df['CryoSleep'], False)
    
    df.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)
    
    df['Cost'] = df['RoomService'] + df['FoodCourt'] + df['ShoppingMall'] + df['Spa'] + df['VRDeck']
    
    list_to_scale = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cost']
    for i in list_to_scale:
        df[i] = scale_columns(df, i)
        
    list_to_hot_encode = ['HomePlanet', 'Destination']
    for i in list_to_hot_encode:
        hot_encoded, unique_val_count = one_hot_encode(df, i)
        new_columns = []
        for j in range(unique_val_count):
            new_columns.append(f'{i}-{j}')
        df[new_columns] = hot_encoded    
    df.drop(["HomePlanet", "Destination"], axis=1, inplace=True)
    
    if not split_target:
        return df, passenger_id
    
    target = df["Transported"]
    df_temp = df.drop("Transported", axis=1)
    
    return df_temp, target, passenger_id

In [56]:
train_modified, train_pass = dataset_transform(train, False)
test_modified, test_pass = dataset_transform(test, False)

In [57]:
train_modified.head()

,Transported,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cost,HomePlanet-0,HomePlanet-1,HomePlanet-2,Destination-0,Destination-1,Destination-2
0,0,0,0.493671,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,1.0
1,1,0,0.303797,0,0.007608,0.000302,0.001064,0.024500,0.001823,0.020452,1.0,0.0,0.0,0.0,0.0,1.0
2,0,0,0.734177,1,0.003001,0.119948,0.000000,0.299670,0.002030,0.288521,0.0,1.0,0.0,0.0,0.0,1.0
3,0,0,0.417722,0,0.000000,0.043035,0.015793,0.148563,0.007997,0.143830,0.0,1.0,0.0,0.0,0.0,1.0
4,1,0,0.202532,0,0.021149,0.002348,0.006428,0.025214,0.000083,0.030317,1.0,0.0,0.0,0.0,0.0,1.0


In [58]:
train_pass

0       0001_01
1       0002_01
2       0003_01
3       0003_02
4       0004_01
         ...   
8688    9276_01
8689    9278_01
8690    9279_01
8691    9280_01
8692    9280_02
Name: PassengerId, Length: 8693, dtype: object

In [59]:
test_modified.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cost,HomePlanet-0,HomePlanet-1,HomePlanet-2,Destination-0,Destination-1,Destination-2
0,1,0.341772,0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,1.0
1,0,0.240506,0,0.000000,0.000356,0.00000,0.142260,0.000000,0.084120,1.0,0.0,0.0,0.0,0.0,1.0
2,1,0.392405,0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0
3,0,0.481013,0,0.000000,0.263206,0.00000,0.009121,0.026266,0.220341,0.0,1.0,0.0,0.0,0.0,1.0
4,0,0.253165,0,0.000865,0.000000,0.07658,0.000000,0.000000,0.019159,1.0,0.0,0.0,0.0,0.0,1.0


In [60]:
train_file = "train_modified.csv"
test_file = "test_modified.csv"
train_modified.to_csv(f'./{train_file}', index=False, header=False)
test_modified.to_csv(f'./{test_file}', index=False, header=False)

boto3.Session().resource("s3").Bucket(bucket).Object(os.path.join(prefix, f'train/{train_file}')).upload_file(train_file)
boto3.Session().resource("s3").Bucket(bucket).Object(os.path.join(prefix, f'test/{test_file}')).upload_file(test_file)


In [77]:
from sklearn.model_selection import train_test_split

train_temp = pd.read_csv(f's3://{bucket}/{prefix}/train.csv')

hyper_train, hyper_target, hyper_passenger = dataset_transform(train_temp, True)

h_train, h_test, h_train_target, h_test_target = train_test_split(hyper_train,
                                                                  hyper_target,
                                                                  test_size=0.3,
                                                                  random_state=0)

In [65]:
space = {
    'max_depth': hp.quniform("max_depth", 3, 18, 1),
    'gamma': hp.uniform ('gamma', 0,9),
    'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
    'reg_lambda' : hp.uniform('reg_lambda', 0,1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
    'n_estimators': 180,
    'seed': 0
}

In [78]:
def objective(space):
    clf=xgb.XGBClassifier(
        n_estimators =space['n_estimators'], 
        max_depth = int(space['max_depth']), 
        gamma = space['gamma'],
        reg_alpha = int(space['reg_alpha']),
        min_child_weight=int(space['min_child_weight']),
        colsample_bytree=int(space['colsample_bytree']),
        early_stopping_rounds=10,
        eval_metric="auc",
        objective="binary:logistic"
    )
    
    evaluation = [(h_train, h_train_target), (h_test, h_test_target)]
    
    clf.fit(h_train, 
            h_train_target,
            eval_set=evaluation, 
            verbose=False)
    

    pred = clf.predict(h_test)
    accuracy = accuracy_score(h_test_target, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [79]:
trials = Trials()

best_hyperparams = fmin(
    fn = objective,
    space = space,
    algo = tpe.suggest,
    max_evals = 100,
    trials = trials
)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.003264 seconds
INFO:hyperopt.tpe:TPE using 0 trials


SCORE:                                                 
0.7335122699386503                                     
  1%|          | 1/100 [00:00<01:31,  1.08trial/s, best loss: -0.7335122699386503]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.010085 seconds
INFO:hyperopt.tpe:TPE using 1/1 trials with best loss -0.733512


SCORE:                                                                            
0.7266104294478528                                                                
  2%|▏         | 2/100 [00:01<01:17,  1.26trial/s, best loss: -0.7335122699386503]

...

SCORE:                                                                             
0.7304447852760736                                                                 
 98%|█████████▊| 98/100 [00:50<00:01,  1.81trial/s, best loss: -0.7611196319018405]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.001842 seconds
INFO:hyperopt.tpe:TPE using 98/98 trials with best loss -0.761120


SCORE:                                                                             
0.7266104294478528                                                                 
 99%|█████████▉| 99/100 [00:51<00:00,  1.91trial/s, best loss: -0.7611196319018405]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.002003 seconds
INFO:hyperopt.tpe:TPE using 99/99 trials with best loss -0.761120


SCORE:                                                                             
0.7254601226993865                                                                 
100%|██████████| 100/100 [00:51<00:00,  1.94trial/s, best loss: -0.7611196319018405]


In [80]:
best_hyperparams['max_depth'] = best_hyperparams['max_depth'].astype(float).astype(int)

print("The best hyperparameters are : ","\n")
print(best_hyperparams)

The best hyperparameters are :  

{'colsample_bytree': 0.5884648549513093, 'gamma': 1.038630608684157, 'max_depth': 3, 'min_child_weight': 0.0, 'reg_alpha': 41.0, 'reg_lambda': 0.9837495930439122}


In [81]:
from sagemaker.inputs import TrainingInput

input_train = TrainingInput(
    s3_data=f's3://{bucket}/{prefix}/train/{train_file}',
    content_type="csv"
)

container = sagemaker.image_uris.retrieve("xgboost", region, "1.7-1")

hyperpar = {
    'objective':'binary:logistic',
    'eval_metric':'error',
    'num_round':'100'
}

hyperpar.update(best_hyperparams)

print(hyperpar)

output_path = f's3://{bucket}/{prefix}/output'

estimate = sagemaker.estimator.Estimator(
    image_uri=container,
    hyperparameters=hyperpar,
    role=role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path=output_path,
    sagemaker_session=sess)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


{'objective': 'binary:logistic', 'eval_metric': 'error', 'num_round': '100', 'colsample_bytree': 0.5884648549513093, 'gamma': 1.038630608684157, 'max_depth': 3, 'min_child_weight': 0.0, 'reg_alpha': 41.0, 'reg_lambda': 0.9837495930439122}


In [82]:
estimate.fit({"train": input_train})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-06-01-06-30-21-798


2023-06-01 06:30:22 Starting - Starting the training job...
2023-06-01 06:30:47 Starting - Preparing the instances for training......
2023-06-01 06:31:56 Downloading - Downloading input data...
2023-06-01 06:32:15 Training - Downloading the training image......
2023-06-01 06:33:32 Uploading - Uploading generated training model
2023-06-01 06:33:32 Completed - Training job completed
[2023-06-01 06:33:16.213 ip-10-0-161-55.eu-west-1.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-06-01 06:33:16.287 ip-10-0-161-55.eu-west-1.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-06-01:06:33:16:INFO] Imported framework sagemaker_xgboost_container.training
[2023-06-01:06:33:16:INFO] Failed to parse hyperparameter eval_metric value error to Json.
Returning the value itself
[2023-06-01:06:33:16:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2023-06-01:06:33:16:INFO] No GPU

In [83]:
batch_input = f's3://{bucket}/{prefix}/test/{test_file}'
batch_output = f's3://{bucket}/{prefix}/batch-inference'


transformer = estimate.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    strategy="MultiRecord",
    assemble_with="Line",
    accept="text/csv",
    output_path=batch_output
)

transformer.transform(batch_input, content_type="text/csv", split_type="Line")
transformer.wait()

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-06-01-06-35-04-212
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2023-06-01-06-35-05-003


...............................[2023-06-01:06:40:14:INFO] No GPUs detected (normal if no gpus installed)
[2023-06-01:06:40:14:INFO] No GPUs detected (normal if no gpus installed)
[2023-06-01:06:40:14:INFO] nginx config: 
[2023-06-01:06:40:14:INFO] No GPUs detected (normal if no gpus installed)
[2023-06-01:06:40:14:INFO] No GPUs detected (normal if no gpus installed)
[2023-06-01:06:40:14:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy

In [84]:
!aws s3 cp --recursive $transformer.output_path ./

download: s3://test-sagemaker-examples-1357942113492/DEMO_Spaceship/batch-inference/test_modified.csv.out to ./test_modified.csv.out


In [85]:
predictions = pd.read_csv("test_modified.csv.out", header=None)

# test_pass.drop(["Survived"], axis=0, inplace=True)
# test_pass["Survived"] = [round(float(value)) for value in predictions.values]

predictions = pd.DataFrame([bool(round(float(x))) for x in predictions.values])
predictions.columns = ["Transported"]

result = pd.concat([test_pass, predictions], axis=1)

# result.drop(result.columns[0], axis=1, inplace=True)

print(result)

result.to_csv("results.csv", index=False)

     PassengerId  Transported
0        0013_01         True
1        0018_01        False
2        0019_01         True
3        0021_01         True
4        0023_01         True
...          ...          ...
4272     9266_02         True
4273     9269_01        False
4274     9271_01         True
4275     9273_01         True
4276     9277_01         True

[4277 rows x 2 columns]
